In [32]:
%matplotlib inline
import numpy as np

from astropy import units as u
# from astropy.constants import c
from astropy.constants import k_B, G

In [33]:
%%javascript
MathJax.Hub.Config({
    TeX: { equationNumbers: { autoNumber: "AMS" } }
});

<IPython.core.display.Javascript object>

As discussed in our previous telecon, we can use equation 10.19 from Stahler & Palla's book
\begin{equation}
\delta u = \delta u_i \left(1 - \frac{i\omega}{n_i\langle \sigma_{in} u_i' \rangle}\right)^{-1}~,
\label{eq:1}
\end{equation}
where 
$n_i\langle \sigma_{in} u_i' \rangle$
is "the frequency with which a given natural atom or molecule is struck by ions", $\delta u$ is the perturbation on the neutral's velocity, and $\delta u_i$ is the perturbation on the ion's velocity.

The other relation to use is equation 10.21 from Stahler & Palla's book
\begin{equation}
\frac{\omega^2}{k^2} = \frac{B_o^2}{4\pi \rho_o} 
                       \left(1 - \frac{i\omega}{n_i\langle \sigma_{in} u_i' \rangle}\right)~,
\label{eq:2}
\end{equation}
which relates the $\omega$ and $k$.

Finally, equation 10.17 from Stahler & Palla's book relates the perturbation in the magnetic field as
\begin{equation}
\delta u_i = -\frac{\omega}{k} \frac{\delta B}{B_o}~.
\label{eq:3}
\end{equation}

We estimate the velocity perturvations as the velocity dispersions, $\delta u =\sqrt{2} \sigma_v$, derived from the spectral lines and therefore rewrite equation \ref{eq:1} as 
\begin{equation}
\left| \frac{\delta u_i}{\delta u} \right|^2 =
1 + \left(\frac{\omega}{n_i\langle \sigma_{in} u_i' \rangle}\right)^{2}
\approx \left(\frac{\sigma_v({\rm N_2H^+})}{\sigma_v({\rm NH_3})}\right)^2~.
\label{eq:4}
\end{equation}

Now, equation \ref{eq:2} can also be rewritten as
\begin{equation}
\left|\frac{\omega^2}{k^2}\right| = \left(\frac{B_o^2}{4\pi \rho_o}\right)
                       \left[1 + \left(\frac{\omega}{n_i\langle \sigma_{in} u_i' \rangle}\right)^2\right]^{1/2}
\approx \frac{B_o^2}{4\pi \rho_o}
\frac{\sigma_v({\rm N_2H^+})}{\sigma_v({\rm NH_3})}~.
\label{eq:5}
\end{equation}

Combining equations \ref{eq:3}, \ref{eq:4} and \ref{eq:5} we obtain
$$
\frac{\delta B}{B_o} = \sqrt{\sigma_v({\rm N_2H^+}) \sigma_v({\rm NH_3})}
\frac{\sqrt{8\pi \rho_o}}{B_o}
$$
or
$$
\delta B= \sqrt{8\pi \rho_o} \sqrt{\sigma_v({\rm N_2H^+}) \sigma_v({\rm NH_3})}
$$

The term $n_i$ is estimated using the ionization degree from Caselli et al. (2002)
$$
x(e) = \frac{n_i}{n({\rm H_2})} = 5.2\times 10^{-6} \left( \frac{n({\rm H_2})}{{\rm cm}^{-3}}\right)^{-0.56} 
$$
or McKee (1989)
$$
x(e) = 1.3\times 10^{-5} \left( \frac{n({\rm H_2})}{{\rm cm}^{-3}}\right)^{-0.5} 
$$
and the term
$$
\langle \sigma_{in} u_i' \rangle \approx 10^{-9} {\rm cm^3 s^{-1}}
$$
is approximated by the Langevin term.

Using the relation 
$$
\lambda = \frac{2\pi}{k} = \frac{2\pi}{\omega} \frac{\omega}{k}
$$ 
and that 
$$
\frac{1}{\omega} = \frac{1}{n_i \langle \sigma_{in} u_i' \rangle}
\sqrt{\frac{\sigma_v({\rm NH_3})^2}{\sigma_v({\rm N_2H^+})^2-\sigma_v({\rm NH_3})^2}}
$$
then we can write
\begin{equation}
\lambda = \sqrt{\frac{\pi}{\rho_o}}  \frac{B_o}{n_i \langle \sigma_{in} u_i' \rangle} 
\sqrt{\frac{\sigma_v({\rm NH_3}) \sigma_v({\rm N_2H^+})}{\sigma_v({\rm N_2H^+})^2 - \sigma_v({\rm NH_3})^2}}
\end{equation}

In [27]:
def sigma_thermal(mu_mol, tk=10*u.K):
    """
    Returns the sound speed for temperature Tk and molecular weight mu.
    This is also used to determine the thermal velocity dispersion of
    a molecular transition.

    """
    return np.sqrt(k_B * tk/(mu_mol * u.u)).to(u.km/u.s)


def density_ion(dens_all, do_Caselli=True):
    """
    1.3e-5 x n(H2)^{0.5}  (from McKee 1989) or 5.2e-6 x n*H2)^{0.44} 
    """
    if do_Caselli:
        xe = 5.2e-6 * (dens_all/(u.cm**-3))**-0.56
    else:
        xe = 1.3e-5 * (dens_all/(u.cm**-3))**-0.5
    return xe*dens_all


def get_omega(sigma_ion=0.1*u.km/u.s, sigma_neutral=0.08*u.km/u.s, density=1e6/u.cm**3, do_Caselli=True):
    """
    """
    n_i = density_ion(density, do_Caselli=do_Caselli)
    return (sig_in_v_i * n_i * np.sqrt((sigma_ion/sigma_neutral)**2 - 1)).decompose()


def get_wavelength(Bfield=100*u.uG, sigma_ion=0.1*u.km/u.s, sigma_neutral=0.08*u.km/u.s, 
                   n_H2=1e6/u.cm**3, do_Caselli=True):
    """
    """
    rho0 = (n_H2 * u.u * 2.8).cgs
    n_i = density_ion(n_H2, do_Caselli=do_Caselli).cgs
    my_wave = np.sqrt(np.pi/rho0) * (Bfield.to(u.G)/(n_i*sig_in_v_i)) * np.sqrt(sigma_ion.cgs*sigma_neutral.cgs/(sigma_ion.cgs**2 - sigma_neutral.cgs**2))
    return (my_wave.value) * u.cm


def alven_speed(Bfield=100*u.G, n_H2=1e6/u.cm**3):
    """
    """
    rho_o = (n_H2 * u.u * 2.8).cgs
    V_a = Bfield.to(u.G)/np.sqrt(4*np.pi*rho_o)
    return (V_a.value)*(u.cm/u.s)


def delta_B(Bfield=100*u.uG, sigma_ion=0.1*u.km/u.s, sigma_neutral=0.08*u.km/u.s, 
                   n_H2=1e6/u.cm**3):
    rho_o = (n_H2 * u.u * 2.8).cgs
    delta_B_Bo = np.sqrt(sigma_ion * sigma_neutral) * np.sqrt(8 * np.pi * rho_o) / Bfield.to((u.g/u.cm)**(1/2)/u.s, equivalencies=equiv_B)
    return delta_B_Bo.decompose()


sig_in_v_i = 1e-9*u.cm**3/u.s
gauss_B = (u.g/u.cm)**(0.5)/u.s
equiv_B = [(u.G, gauss_B, lambda x: x, lambda x: x)]

B_o = np.array([100, 150, 300.]) * u.uG
c_sound = sigma_thermal(2.38, tk=10*u.K).cgs
sigma_ion = 0.62 * c_sound
sigma_neutral = 0.47 * c_sound
n_H2 = 7e4 / u.cm**3
rho_o = (7e4 / u.cm**3 * u.u * 2.8).cgs

In [28]:
delta_B_Bo = delta_B(Bfield=B_o, sigma_ion=sigma_ion, n_H2=n_H2)#sigma_neutral=sigma_neutral,   np.sqrt(sigma_ion*sigma_neutral * 8*np.pi * rho_o) / B_o.to((u.g/u.cm)**(1/2)/u.s, equivalencies=equiv_B))
print(delta_B_Bo)

[0.275377   0.18358466 0.09179233]


In [29]:
delta_B = delta_B_Bo * B_o#((np.sqrt(2*sigma_ion*sigma_neutral * 4*np.pi * rho_o) / B_o) * B_o).to(u.uG, equivalencies=equiv_B)
print(delta_B)

[27.53769969 27.53769969 27.53769969] uG


In [6]:
density_ion(n_H2, do_Caselli=True)

<Quantity 0.00070444 1 / cm3>

In [7]:
density_ion(n_H2, do_Caselli=False)

<Quantity 0.00343948 1 / cm3>

This corresponds to a variation of $27\,\mu$G, which corresponds between 18 to 27\% of the field.

If we want to estimate the wavelength then we will need to estimate 
$n_i\langle \sigma_{in} u_i' \rangle$ to derive $\omega$ and then obtain $k$.

We use the velocity dispersions for ions and neutrals, the mean density of the coherent core, and the magnetic field strengths.

In [30]:
(get_wavelength(Bfield=B_o, sigma_ion=sigma_ion, sigma_neutral=sigma_neutral, n_H2=n_H2)).to(u.pc)

<Quantity [0.19081443, 0.28622165, 0.5724433 ] pc>

In [31]:
(get_wavelength(Bfield=B_o, sigma_ion=sigma_ion, sigma_neutral=sigma_neutral, n_H2=n_H2, do_Caselli=False)).to(u.pc)#, equivalencies=equiv_B)

<Quantity [0.03908097, 0.05862145, 0.11724291] pc>

Therefore, the wavelength is between 0.04 pc and 0.29 pc, where the spread is dominated by the uncertainty on the ionization fraction. The size of the coherent region is 0.34 pc (in diameter).

And for the same density and magnetic field strength we estimate the Alven speed inside the dense coherent core.

In [11]:
alven_speed(n_H2=n_H2, Bfield=B_o).to(u.km/u.s)

<Quantity [0.49447289, 0.74170933] km / s>

In [12]:
alven_speed(n_H2=n_H2/10, Bfield=B_o).to(u.km/u.s)

<Quantity [1.56366057, 2.34549085] km / s>

## Estimating MHD wave outside the coherent region

We have a good estimate of the amount of turbulence outside the coherent region. The mean velocity dispersion from NH3 outside the coherent region of Barnard 5 is 0.34 km/s (Pineda et al. 2010). We estimate the kinetic temperature in this outer region as 13 K (based on comparison between cores and cloud from Choudhury et al. in prep.).

If we assume that an upper limit of the MHD wave is the observed non-thermal component, then we can estimate the scale of this turbulence using Larson's law
$$
R = 1\,{\rm pc} \left(\frac{\sigma_{nt}}{1 {\rm km\, s^{-1}} /\sqrt{8 \log{2}}}\right)^2~.
$$

In this way we estimate an upper limit for the wavelength of 0.6 pc for the wave, which is larger than the coherent core scale (0.34 pc in diameter).

In [13]:
def size_sigma_v(sigma_v=1.0*u.km/u.s):
    """
    """
    sigma_v0 = 1.*u.km/u.s / np.sqrt(8*np.log(2))
    return  1.*u.pc * (sigma_v/sigma_v0)**2


sigma_v_ammo_super = 0.34*u.km/u.s
sigma_nt_ammo_super = np.sqrt(sigma_v_ammo_super**2 - sigma_thermal(17, tk=13*u.K)**2)

In [14]:
size_sigma_v(sigma_v=sigma_nt_ammo_super)

<Quantity 0.60576562 pc>

In [18]:
n_H2_super = (n_H2 * sigma_neutral**2 /sigma_nt_ammo_super**2).to(1e3*u.cm**-3)
print(n_H2_super)

4.9449489500449 1000 / cm3


## Magnetic field estimate
### B-field estimate from filament properties
Following the discussion with Steve, I will estimate the magnetic field using the following:
$$
\frac{M}{L} = 2\frac{c_s^2}{G}\left( 1+\beta^{-1}\right) 
\frac{\left(R_{top}/R_{flat}\right)^2}{1+\left(R_{top}/R_{flat}\right)^2}
$$
and
$$
\beta = \frac{P_{gas}}{P_{B}}
= \frac{8\pi \rho c_s^2}{B^2}~.
$$
Then we estimate the magnetic field strength as:
$$
B = \sqrt{8\pi \rho} c_s 
\left[ 
\frac{G}{2c_s^2} \frac{M}{L}
\left(\frac{1+\xi_R^2}{\xi_R^2}\right) -1 \right]^{1/2}
$$
where $\xi_R\equiv R_{top}/R_{flat}$

### Fitted density (column density) profile
Anika fitted the derived column densities with the following function
$$
    \Sigma(r) = A_p \frac{\Sigma_0}{\left[1+\left(r/R_{flat}\right)^2\right]^{(p-1)/2}}~,
$$
where $\Sigma_0$ is the peak column density, $A_p= \frac{1}{\cos i} \int_{-\inf}^{\inf} \frac{du}{(1+u^2)^{p/2}}$ is a geometrical factor for $p>1$ (see Arzoumanian et al. 2011), and $R_{flat}$ is the flattening radius.
This column density profile is equivalent to fitting a density profile of
$$
    n(r) = \frac{n_0}{\left[1+\left(r/R_{flat}\right)^2\right]^{p/2}}~,
$$
where $n_0$ is the central density.

The calculations from Anika are for the dense filaments, not the core, and therefore the filament central density is $10^6$ cm$^{-3}$, and it implies a magnetic field of 384 $\mu$G to stabilize the filament. However, the equation used is derived for a $p=2$ profile (I think).

In [19]:
def get_beta(R_top=1.*u.pc, R_flat=0.1*u.pc, Tk=10*u.K, m_l=1*u.Msun/u.pc):
    left_side = G * m_l.cgs / (2*sigma_thermal(2.37, tk=Tk)**2).cgs * (1+(R_top/R_flat)**2)/(R_top/R_flat)**2
    beta = 1./(left_side - 1)
    return beta


def get_B_form_beta(beta=1.0, Tk=10*u.K, n_H2=1e6/u.cm**3):
    """
    """
    rho0 = (n_H2 * u.u * 2.8).cgs
    c_s = sigma_thermal(2.37, tk=Tk).cgs
    P_gas = rho0 * c_s**2
    Bfield = np.sqrt(8*np.pi * P_gas / beta)
    return (Bfield).to(u.uG, equivalencies=equiv_B)
    

In [20]:
beta_fil1 = get_beta(m_l=60*u.Msun/u.pc, R_flat=2844*u.au, R_top=2*2844*u.au)

In [21]:
get_B_form_beta(beta=beta_fil1, Tk=10*u.K, n_H2=1e6/u.cm**3)

<Quantity 384.02287033 uG>

In [22]:
beta_fil1

<Quantity 0.27798326>

In [25]:
beta_fil1 = get_beta(m_l=60*u.Msun/u.pc, R_flat=2844*u.au, R_top=5*2844*u.au)

In [26]:
get_B_form_beta(beta=beta_fil1, Tk=10*u.K, n_H2=1e6/u.cm**3)

<Quantity 340.3098813 uG>